Make Functions Work for Classification Export 41


In [1]:
import os
import pandas as pd
from pandasgui import show
import numpy as np
from fancyimpute import IterativeImputer
from datetime import datetime

In [38]:
# <41-1> Function to process Export 41 noise data: Adding Time variables_v230601
    # Remove noise_event_laeq_primary_detected_certainty, and all other units(blank)

def add_time_41(file_path):
    import os
    import pandas as pd
    
    file_list = []
    for file in os.listdir(file_path):
        file_list.append(os.path.join(file_path, file))
    print(file_list) # To be deleted after final confirmation

    # Create new folder path
    new_folder = os.path.join(file_path, 'withTime')  
    # Create the concatenated' folder if it doesn't exist
    os.makedirs(new_folder, exist_ok=True)

    
    for file in file_list:
        # Read Files
        df = pd.read_csv(file, delimiter=';')

        # Convert 'result_timestamp' column to datetime format
        df['result_timestamp'] = pd.to_datetime(df['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')

        # create new columns for year, month, day, hour, minute, and weekday
        df['year'] = df['result_timestamp'].dt.year
        df['month'] = df['result_timestamp'].dt.month
        df['day'] = df['result_timestamp'].dt.day
        df['hour'] = df['result_timestamp'].dt.hour
        df['minute'] = df['result_timestamp'].dt.minute
        df['10min'] = df['minute'].apply(lambda x: int(x/10)*10)
        df['weekday'] = df['result_timestamp'].dt.weekday
        
        # Create the new DataFrame with the desired columns
        new_df = pd.DataFrame()
        new_df['description'] = df['description']
        new_df['year'] = df['year']
        new_df['month'] = df['month']
        new_df['day'] = df['day']
        new_df['hour'] = df['hour']
        new_df['weekday'] = df['weekday']
        new_df['10min'] = df['10min']
        new_df['noise_event_laeq_model_id'] = df['noise_event_laeq_model_id']
        new_df['noise_event_laeq_primary_detected_certainty'] = df['noise_event_laeq_primary_detected_certainty']
        new_df['noise_event_laeq_primary_detected_class'] = df['noise_event_laeq_primary_detected_class']
        
        # Define the new file path
        file_name = os.path.basename(file)
        new_file_path = os.path.join(new_folder, file_name[:-4]+'_withTime.csv')
        
        # Save the new DataFrame as a CSV file
        new_df.to_csv(new_file_path, index=False)
    

In [39]:
# <41-1> Example Function to process Export 41 noise data_v230601
file_path = r'C:\Users\LIE\MDA Project\Sound in Street\export_41_test'
add_time_41(file_path)

['C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\csv_results_41_255439_mp-01-naamsestraat-35-maxim.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\csv_results_41_255440_mp-02-naamsestraat-57-xior.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\csv_results_41_255441_mp-03-naamsestraat-62-taste.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\csv_results_41_255442_mp-05-calvariekapel-ku-leuven.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\csv_results_41_255443_mp-06-parkstraat-2-la-filosovia.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\csv_results_41_255444_mp-07-naamsestraat-81.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\csv_results_41_280324_mp08bis---vrijthof.csv']


In [40]:
# <41-2> Function to create a timestamp column to each site of the Export41, then export_v230601
    # There are no record in these 2 sites:
    # csv_results_41_255445_mp-08-kiosk-stadspark & csv_results_41_303910_mp-04-his-hears
    # Therefore they were removed. Other the function will not work.
    
def  create_timestamp(min_time, max_time, file_path):
    import pandas as pd
    from datetime import datetime
    import os
    # Define min and max time
    min_time = pd.to_datetime(min_time)
    max_time = pd.to_datetime(max_time)
    # Create a new dataframe with all possible combinations of time variables
    time_df = pd.DataFrame({'datetime': pd.date_range(min_time, max_time, freq='10min')})

    # Create a list of all the files in the directory
    file_list = []
    for file in os.listdir(file_path):
        file_list.append(os.path.join(file_path, file))
    print(file_list) # To be deleted after final confirmation

    # Create new folder path
    new_folder = os.path.join(file_path, 'addTimestamp')  
    # Create the concatenated' folder if it doesn't exist
    os.makedirs(new_folder, exist_ok=True)
    
    for file in os.listdir(file_path):
        file_list.append(os.path.join(file_path, file))
    # Read Files and merge the files with TimeFrame dataframe.
    for file in file_list:    
        df = pd.read_csv(file)
        df['datetime'] = df.apply(lambda x: datetime(x['year'], x['month'], x['day'], x['hour'], int(x['10min'])), axis=1)
        merged_df = pd.merge(time_df, df, how='outer', on='datetime')

        # Define the new file path
        file_name = os.path.basename(file)
        new_file_path = os.path.join(new_folder, file_name[:-4]+'_addTimestamp.csv')
       
        merged_df.to_csv(new_file_path, index=False)

In [41]:
# <41-2> Example of Function to create a timestamp column, then export_v230601
file_path = r'C:\Users\LIE\MDA Project\Sound in Street\export_41_test\withTime'
create_timestamp('2022-01-01 00:00', '2023-01-01 00:00', file_path)

['C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\csv_results_41_255439_mp-01-naamsestraat-35-maxim_withTime.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\csv_results_41_255440_mp-02-naamsestraat-57-xior_withTime.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\csv_results_41_255441_mp-03-naamsestraat-62-taste_withTime.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\csv_results_41_255442_mp-05-calvariekapel-ku-leuven_withTime.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\csv_results_41_255443_mp-06-parkstraat-2-la-filosovia_withTime.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\csv_results_41_255444_mp-07-naamsestraat-81_withTime.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\csv_results_41_280324_mp08bis---vrijthof_withTime.csv']


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\addTimestamp'

In [ ]:
#<41-3> Join 41 and Weather_v230601
def join_weather41(file_path, weatherFile):
    import os
    import pandas as pd
    from datetime import datetime

    df_weather = pd.read_csv(weatherFile)
    # Rename the column in df_weather to match the column name in df
    df_weather = df_weather.rename(columns={'DATE_BRU': 'datetime'})
    df_weather['datetime'] = df_weather['datetime'].str[:-6]
    file_list = []
    for file in os.listdir(file_path):
        file_list.append(os.path.join(file_path, file))
    print(file_list) # To be deleted after final confirmation
    
    # Create new folder path
    new_folder = os.path.join(file_path, 'withWeather')  
    # Create the concatenated' folder if it doesn't exist
    os.makedirs(new_folder, exist_ok=True)
    
    for file in file_list:
        df = pd.read_csv(file)
        merged_df = pd.merge(df, df_weather, on='datetime', how='outer')
        print(merged_df['noise_event_laeq_primary_detected_class'].value_counts()) # To be deleted after final confirmation
        
        # Remove the row 'noise_event_laeq_primary_detected_class' with missing values
        merged_df = merged_df.dropna(subset=['noise_event_laeq_primary_detected_class'])
        print(merged_df['noise_event_laeq_primary_detected_class'].value_counts()) # To be deleted after final confirmation
        
        # Define the new file path and save files
        file_name = os.path.basename(file)
        new_file_path = os.path.join(new_folder, file_name[:-4]+'_withWeather.csv')       
        merged_df.to_csv(new_file_path, index=False)
    return new_file_path

In [42]:
#<41-3> Example Join 41 and Weather_v230601
file_path = r'C:\Users\LIE\MDA Project\Sound in Street\export_41_test\withTime\addTimestamp'
weatherFile = r'C:\Users\LIE\MDA Project\Weather\lc_2022_avgbyid_timecoverted.csv'
join_weather41(file_path, weatherFile)

['C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\addTimestamp\\csv_results_41_255439_mp-01-naamsestraat-35-maxim_withTime_addTimestamp.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\addTimestamp\\csv_results_41_255440_mp-02-naamsestraat-57-xior_withTime_addTimestamp.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\addTimestamp\\csv_results_41_255441_mp-03-naamsestraat-62-taste_withTime_addTimestamp.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\addTimestamp\\csv_results_41_255442_mp-05-calvariekapel-ku-leuven_withTime_addTimestamp.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\addTimestamp\\csv_results_41_255443_mp-06-parkstraat-2-la-filosovia_withTime_addTimestamp.csv', 'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\addTimestamp\\csv_results_41_255444_mp-07-naamsestraat-81_withTime_addTimestamp.csv', 'C:\\Users

'C:\\Users\\LIE\\MDA Project\\Sound in Street\\export_41_test\\withTime\\addTimestamp\\withWeather\\csv_results_41_280324_mp08bis---vrijthof_withTime_addTimestamp_withWeather.csv'

In [43]:
# <41-4> Function to cancatenate the Export41+weather files into one file for machine learning or visualisation_v230601

def concatenate_files(file_path):
    import os
    # Create a list of all the files in the directory
    file_list = []
    for file in os.listdir(file_path):
        file_list.append(os.path.join(file_path, file))

    # Create new folder path
    new_folder = os.path.join(file_path, 'concatenated')  
    # Create the concatenated' folder if it doesn't exist
    os.makedirs(new_folder, exist_ok=True)
        
    # Read files, concatenate, then export.
    import pandas as pd
    merged_df = pd.DataFrame()
    for file in file_list:    
        df = pd.read_csv(file)
        merged_df = pd.concat([merged_df, df])
    # Add "Humidity>80" column
    merged_df['Humidity>80'] = (merged_df['LC_HUMIDITY'] >= 80).astype(int)

    # Define the new file path and export
    file_name = os.path.basename(file)
    new_file_path = os.path.join(new_folder, file_name[:-4]+'_concatenated.csv')
    merged_df.to_csv(new_file_path, index=False)

In [44]:
# <41-4> Example Function to cancatenate the Export41+weather files for machine learning or visualisation_v230601
file_path = r'C:\Users\LIE\MDA Project\Sound in Street\export_41_test\withTime\addTimestamp\withWeather'
concatenate_files(file_path)